<a href="https://colab.research.google.com/github/Miseadolch/e-com/blob/Maria/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
import datetime as dt


# функция для столбца времени суток
def time(row):
    times = {
        'morning': [dt.time(hour=6), dt.time(hour=9, minute=59)],
        'day': [dt.time(hour=10), dt.time(hour=16, minute=59)],
        'evening': [dt.time(hour=17), dt.time(hour=21, minute=59)],
        'night': [dt.time(hour=22), dt.time(hour=5, minute=59)]
    }
    d = dt.date(2021, 10, 12)
    a = pd.to_datetime(row.session_start, format='%Y-%m-%d %H:%M:%S').time()
    b = pd.to_datetime(row.session_end, format='%Y-%m-%d %H:%M:%S').time()
    for i in times.keys():
        if a >= times[i][0] and b <= times[i][1]:
            return i
        if times[i][0] <= a < times[i][1] and b > times[i][1]:
            if row.sessiondurationsec / 2 <= (
                dt.datetime.combine(d, times[i][1]) - dt.datetime.combine(
                    d, a)).seconds:
                return i
        if a <= times[i][0] and b >= times[i][1]:
            return i
        if a < times[i][0] and times[i][0] < b <= times[i][1]:
            if row.sessiondurationsec / 2 <= (
                dt.datetime.combine(d, b) - dt.datetime.combine(
                    d, times[i][0])).seconds:
                return i
        if i == 'night' and (a <= times[i][1] and b <= times[i][1] or
                             a >= times[i][0] and b >= times[i][0]):
            return i


df = pd.read_csv('res.csv', sep=',', encoding='cp1251', skipinitialspace=True, decimal='.')
df.columns = map(lambda x: x.lower().replace(" ", "_"), df.columns)
# исправление нецелых в промокоде
df['promo_code'] = df['promo_code'].map(lambda x: 1 if not pd.isna(x) and x else x)
# столбец цены с использованием промокода
df['promo_code_sum'] = df.apply(lambda x: 0.9 * x.revenue if
                                x.promo_code == 1 and not pd.isna(
                                    x.revenue) else x.revenue, axis=1)
# столбец является ли покупателем пользователь
df['payer'] = df['revenue'].apply(lambda x: 1 if not pd.isna(x) else 0)
# функция для столбца времени суток
df['time_of_day'] = df.apply(time, axis=1)
df

,user_id,region,device,channel,session_start,session_end,sessiondurationsec,session_date,month,day,hour_of_day,order_dt,revenue,payment_type,promo_code,revenue_with_promo_code,payer,promo_code_sum,time_of_day
0,529697267522,United States,iPhone,СЃРѕС†РёР°Р»СЊРЅС‹Рµ СЃРµС‚Рё,2019-05-01 00:06:40,2019-05-01 00:07:06,26.0,2019-05-01,5,3,0,2019-05-01 00:06:40,9999.0,Mobile payments,0.0,9999.0,1,9999.0,night
1,601292388085,United States,PC,organic,2019-05-01 06:56:16,2019-05-01 07:09:18,782.0,2019-05-01,5,3,7,NaN,NaN,NaN,NaN,NaN,0,NaN,morning
2,852898876338,United States,Mac,СЃРѕС†РёР°Р»СЊРЅС‹Рµ СЃРµС‚Рё,2019-05-01 04:30:45,2019-05-01 04:34:56,251.0,2019-05-01,5,3,4,NaN,NaN,NaN,NaN,NaN,0,NaN,night
3,998513020664,United States,iPhone,СЃРѕС†РёР°Р»СЊРЅС‹Рµ СЃРµС‚Рё,2019-05-01 18:53:42,2019-05-01 18:57:35,233.0,2019-05-01,5,3,18,NaN,NaN,NaN,NaN,NaN,0,NaN,evening
4,240702200943,United States,Mac,СЃРѕС†РёР°Р»СЊРЅС‹Рµ СЃРµС‚Рё,2019-05-02 14:04:32,2019-05-02 14:09:51,319.0,2019-05-02,5,4,14,NaN,NaN,NaN,NaN,NaN,0,NaN,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,246163171672,United States,iPhone,СЃРѕС†РёР°Р»СЊРЅС‹Рµ СЃРµС‚Рё,2019-10-29 18:12:24,2019-10-29 18:13:39,75.0,2019-10-29,10,2,18,NaN,NaN,NaN,NaN,NaN,0,NaN,evening
999,660140862425,Germany,Android,СЂРµРєР»Р°РјР° Сѓ Р±Р»РѕРіРµСЂРѕРІ,2019-10-30 19:14:02,2019-10-30 19:46:19,1937.0,2019-10-30,10,3,19,NaN,NaN,NaN,NaN,NaN,0,NaN,evening
1000,898253063062,France,iPhone,organic,2019-10-30 03:18:22,2019-10-30 04:43:35,5113.0,2019-10-30,10,3,4,NaN,NaN,NaN,NaN,NaN,0,NaN,night
1001,780224321316,United States,iPhone,СЃРѕС†РёР°Р»СЊРЅС‹Рµ СЃРµС‚Рё,2019-10-31 22:56:17,2019-10-31 23:42:27,2770.0,2019-10-31,10,4,23,NaN,NaN,NaN,NaN,NaN,0,NaN,night
